### BBC

In [2]:
!pip install selenium

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 1.4 MB/s eta 0:00:08
   ---------------------------------------- 0.0/9.7 MB 219.4 kB/s eta 0:00:45
   ---------------------------------------- 0.1/9.7 MB 297.7 kB/s eta 0:00:33
   ---------------------------------------- 0.1/9.7 MB 245.8 kB/s eta 0:00:40
   ---------------------------------------- 0.1/9.7 MB 245.8 kB/s eta 0:00:40
   ---------------------------------------- 0.1/9.7 MB 275.8 kB/s eta 0:00:35
   ---------------------------------------- 0.1/9.7 MB 297.7 kB/s eta 0:00:33
    --------------------------------------- 0.1/9.7 MB 288.8 kB/s eta 0:00:34
    --------------------------------------- 0.2/9.7 MB 316.5 kB/s eta 0:00:31
    -------------

In [ ]:
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'
options = webdriver.EdgeOptions()

# Load company and crypto names
df = pd.read_excel('company_crypto_list.xlsx')  # Your input file with company/crypto names
names = df['Name'].dropna().tolist()  # Get the list of names

# Initialize a list to store article data
article_data = []

# Function to search, click the first link, scrape data, and close the browser
def search_and_scrape(name, news_site):
    service = Service(executable_path=msedgedriver_path)
    driver = webdriver.Edge(service=service, options=options)
    search_query = f"{news_site} {name}"
    search_url = f"https://www.google.com/search?q={search_query}"
    
    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 15)
        
        # Random delay to avoid detection
        time.sleep(random.uniform(8, 12))
        
        # Locate the results and check for "bbc.com"
        results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        found_bbc_link = False

        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, "cite")
                if "bbc.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                    print(f"Found BBC article for '{name}': {link}")
                    result.find_element(By.TAG_NAME, "a").click()
                    found_bbc_link = True
                    break
            except NoSuchElementException:
                continue
        
        if not found_bbc_link:
            print(f"No BBC link found for '{name}' on {news_site}. Exiting search.")
            return

        # Scrape data from the BBC article
        time.sleep(3)
        
        # Click and scrape the first three articles
        for i in range(3):
            try:
                articles = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-testid="liverpool-article"]')))
                if i < len(articles):  
                    article = articles[i]
                    article_title = article.find_element(By.CSS_SELECTOR, 'h2[data-testid="card-headline"]')
                    article_title.click()

                    time.sleep(3)

                    title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#main-content > article > div:nth-child(1) > h1'))).text
                    
                    try:
                        author = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="byline-new"] .sc-2b5e3b35-7'))).text
                    except NoSuchElementException:
                        author = 'NA'  

                    try:
                        date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.sc-2b5e3b35-1.jTEdni time'))).text
                    except NoSuchElementException:
                        date = 'NA'  

                    paragraphs = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[data-component="text-block"] p.sc-eb7bd5f6-0.fYAfXe')))
                    text = ' '.join([para.text for para in paragraphs])
                    
                    article_data.append({
                        'Name': name,
                        'Website': news_site,
                        'Link': driver.current_url,
                        'Title': title,
                        'Author': author,
                        'Date': date,
                        'Text': text
                    })
                    driver.back()
                    time.sleep(2)
                else:
                    print(f"Not enough articles found for '{name}' on {news_site}.")
                    break
            except Exception as e:
                print(f"Error clicking article {i + 1} for '{name}' on {news_site}: {e}")
                continue
    except (TimeoutException, NoSuchElementException) as e:
        print(f"Error for '{name}' on {news_site}: {e}")
    finally:
        driver.quit()

# Main function to process each name
def process_names(names):
    max_threads = 5
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for name in names:
            futures.append(executor.submit(search_and_scrape, name, "BBC"))
        for future in as_completed(futures):
            future.result()

# Run the script
process_names(names)

# Create a DataFrame from the article data and save it to Excel
if article_data:
    df_articles = pd.DataFrame(article_data)
    df_articles.to_excel('bbc_articles.xlsx', index=False)
    print("Data saved to 'bbc_articles.xlsx'.")
else:
    print("No articles scraped.")

print("Google search and scrape process completed.")


Found BBC article for 'Tesla': https://www.bbc.com/news/topics/c8nq32jwjnmt
Data saved to 'bbc_articles.xlsx'.
Google search and scrape process completed.


#### NCC

In [ ]:
import pandas as pd
import random
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Path to Microsoft Edge WebDriver
msedgedriver_path = r'C:\\Users\\mouns\\Documents\\fiverr\\Orders\\Order 16\\scrapping tool\\msedgedriver.exe'

# List of user agents to randomly choose from
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:86.0) Gecko/20100101 Firefox/86.0",
    "Mozilla/5.0 (Linux; Android 10; Pixel 3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.93 Mobile Safari/537.36"
]

# Load company names
df = pd.read_excel('company_crypto_list.xlsx')  # Your input file with company/crypto names
names = df['Name'].dropna().tolist()  # Get the list of names

# Initialize a list to store article data
article_data = []

# Function to save data to Excel
def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)

# Function to perform a random search to mimic human activity
def perform_random_search(driver):
    random_query = f"random search {random.randint(1000, 9999)}"
    search_url = f"https://www.google.com/search?q={random_query}"
    driver.get(search_url)
    time.sleep(random.uniform(2, 4))  # Short delay to simulate browsing

# Function to search for CNN links and scrape data
def search_and_scrape(name):
    service = Service(executable_path=msedgedriver_path)

    def initialize_driver():
        options = webdriver.EdgeOptions()
        options.add_argument(f"user-agent={random.choice(user_agents)}")
        return webdriver.Edge(service=service, options=options)
    
    driver = initialize_driver()
    
    search_query = f"{name} site:cnn.com"
    search_url = f"https://www.google.com/search?q={search_query}"
    
    try:
        print(f"Starting search for '{name}' on CNN")
        driver.get(search_url)
        wait = WebDriverWait(driver, 15)
        
        # Random delay to avoid detection
        time.sleep(random.uniform(8, 12))
        
        # Locate and retrieve the first CNN link
        cnn_links = []
        results = driver.find_elements(By.CSS_SELECTOR, "div.g")
        
        for result in results:
            try:
                cite_element = result.find_element(By.TAG_NAME, "cite")
                if "cnn.com" in cite_element.text:
                    link = result.find_element(By.TAG_NAME, "a").get_attribute("href")
                    if link not in cnn_links:  # Avoid duplicates
                        cnn_links.append(link)
            except NoSuchElementException:
                continue
        
        if not cnn_links:
            print(f"No CNN links found for '{name}'. Exiting search.")
            return
        
        # Process each link with the random search and reset approach
        for link in cnn_links:
            # Close and reopen the browser
            driver.quit()
            driver = initialize_driver()
            
            # Perform a random Google search to avoid detection
            perform_random_search(driver)
            
            # Go to the CNN article link
            driver.get(link)
            time.sleep(random.uniform(8, 12))  # Random delay
            
            print(f"Loading article page: {link}")
            
            try:
                # Extract article title
                title_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1')))
                article_title = title_element.text
                print(f"Article title: {article_title}")
                
                # Extract Author and Date
                try:
                    author = driver.find_element(By.CSS_SELECTOR, 'div.flexccs.displayflex.colorblack.ml3 span.fonth7_semibold').text
                except NoSuchElementException:
                    author = 'NA'
                
                try:
                    date = driver.find_element(By.CSS_SELECTOR, 'div.flexccs.displayflex.colorblack.ml3 span.mobile_fontSize11').text.strip()
                except NoSuchElementException:
                    date = 'NA'
                
                # Extract Article Text
                try:
                    text_elements = driver.find_elements(By.CSS_SELECTOR, 'div.flexcss.displayflex.maxWparent p')
                    article_text = ' '.join([element.text for element in text_elements])
                except NoSuchElementException:
                    article_text = 'NA'
                
                # Store article data
                article_data.append({
                    'Name': name,
                    'Article Title': article_title,
                    'Author': author,
                    'Date': date,
                    'Text': article_text,
                    'Link': link
                })
                
                # Save to Excel after every 10 articles
                if len(article_data) % 10 == 0:
                    save_to_excel(article_data, 'cnn_articles.xlsx')
                    print(f"Saved {len(article_data)} articles to 'cnn_articles.xlsx'.")
            
            except Exception as e:
                print(f"Error extracting details from '{link}': {e}")

    except Exception as e:
        print(f"Error while processing '{name}': {e}")
    finally:
        driver.quit()

# Using ThreadPoolExecutor to process multiple names concurrently
with ThreadPoolExecutor(max_workers=2) as executor:
    futures = {executor.submit(search_and_scrape, name): name for name in names}
    for future in as_completed(futures):
        name = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Error processing '{name}': {e}")

# Final save to ensure any remaining articles are saved
save_to_excel(article_data, 'cnn_articles.xlsx')
print(f"Final save: {len(article_data)} articles saved to 'cnn_articles.xlsx'.")
